<center><img src="https://media-exp1.licdn.com/dms/image/C4E22AQEbIXZiRVkJPQ/feedshare-shrink_2048_1536/0?e=1605139200&v=beta&t=opmJLG_veDflQesF9uJvIhSXbmwPY3DGSlbZo1v-w2k" width="1000"></center>

# Programa de Especialización en Python

## Tema 6: Regresión lineal con h2o
## Caso: conjunto de datos de resistencia a la compresión del hormigón

### Manuel Sigüeñas, M.Sc.(c)
### Prof. Lenguajes de Programación para Ciencia de Datos / Agile Data Scientists / SCRUMStudy Certified Trainer
[msiguenas@socialdata-peru.com](msiguenas@socialdata-peru.com)

### Información del conjunto de datos:

Se proporcionan el nombre de la variable, el tipo de variable, la unidad de medida y una breve descripción. La resistencia a la compresión del hormigón.

### Información del atributo:

**C1:** Cemento (componente 1) - cuantitativo - kg en una mezcla m3 - Variable de entrada

**C2:** Escoria de alto horno (componente 2) - cuantitativa - kg en una mezcla m3 - Variable de entrada

**C3:** Cenizas volantes (componente 3) - cuantitativo - kg en una mezcla m3 - Variable de entrada

**C4:** Agua (componente 4) - cuantitativa - kg en una mezcla m3 - Variable de entrada

**C5:** Superplastificante (componente 5) - cuantitativo - kg en una mezcla m3 - Variable de entrada

**C6:** Agregado grueso (componente 6) - cuantitativo - kg en una mezcla m3 - Variable de entrada

**C7:** Agregado fino (componente 7) - cuantitativo - kg en una mezcla m3 - Variable de entrada

**C8:** Age - cuantitativa - Día (1 ~ 365)

**C9:** Resistencia a la compresión del hormigón - cuantitativa - MPa - Variable de salida


Documentos relevantes:

I-Cheng Yeh, "Modeling of strength of high performance concrete using artificial 
neural networks," Cement and Concrete Research, Vol. 28, No. 12, pp. 1797-1808 (1998).


Fuente:
http://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

- **Paquetes necesarios**:

In [13]:
#!pip uninstall h2o

In [14]:
#pip install http://h2o-release.s3.amazonaws.com/h2o/rel-zahradnik/1/Python/h2o-3.30.0.1-py2.py3-none-any.whl

In [1]:
import h2o

In [2]:
h2o.init (nthreads = -1, max_mem_size = "4g")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\SOCIAL~1\AppData\Local\Temp\tmp4jjy1tct
  JVM stdout: C:\Users\SOCIAL~1\AppData\Local\Temp\tmp4jjy1tct\h2o_SOCIAL_DATA_started_from_python.out
  JVM stderr: C:\Users\SOCIAL~1\AppData\Local\Temp\tmp4jjy1tct\h2o_SOCIAL_DATA_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Bogota
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,H2O_from_python_SOCIAL_DATA_0t16f8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


<a id='part1'></a>
## Importar y preprocesar la data

In [3]:
cement = "D:/Python/2. Nivel II/6/datos/Concrete.csv"
cement_df = h2o.import_file(path=cement)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
cement_df

C1,C2,C3,C4,C5,C6,C7,C8,C9
540,0,0,162,2.5,1040,676,28,79.99
540,0,0,162,2.5,1055,676,28,61.89
332.5,142.5,0,228,0,932,594,270,40.27
332.5,142.5,0,228,0,932,594,365,41.05
198.6,132.4,0,192,0,978.4,825.5,360,44.3
266,114,0,228,0,932,670,90,47.03
380,95,0,228,0,932,594,365,43.7
380,95,0,228,0,932,594,28,36.45
266,114,0,228,0,932,670,28,45.85
475,0,0,228,0,932,594,28,39.29


## Construir data de entrenamiento y de prueba (test)  

In [8]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [9]:
train,test,valid = cement_df.split_frame(ratios=[.8,.15], seed=1234)

In [10]:
name_columns = cement_df.col_names
name_columns

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

## Entrenamiento del Modelo

In [12]:
predictors = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']

response_col = 'C9'
glm_model = H2OGeneralizedLinearEstimator(family="gaussian")
glm_model.train(predictors, #etiquetas de predictores
                response_col, #etiqueta de target (o variable objetivo)
                training_frame= train, #datos de entrenamiento
                validation_frame=valid) #datos de validación

glm Model Build progress: |███████████████████████████████████████████████| 100%


## Predicción del Modelo

In [13]:
predict = glm_model.model_performance(test) #model_performance para realizar la evaluación por métricas

predict


ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 129.1799690194928
RMSE: 11.365736624587639
MAE: 9.116039346265797
RMSLE: 0.38295105900532544
R^2: 0.5271317313566728
Mean Residual Deviance: 129.1799690194928
Null degrees of freedom: 156
Residual degrees of freedom: 148
Null deviance: 44078.167578533365
Residual deviance: 20281.255136060372
AIC: 1228.7561263287218


In [14]:
glm_model.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


predict
31.7135
20.6521
61.4719
37.5977
49.8516
49.0945
24.5655
29.2076
59.0492
39.1119


## Metricas de Regresión

In [15]:
print('MSE train: %.3f, test: %.3f' % (
        glm_model.mse(valid=True),
        predict.mse()
))

MSE train: 118.262, test: 129.180


In [16]:
#RMSE
import math as m
m.sqrt(glm_model.mse(valid=True))

10.874842194663989

In [17]:
m.sqrt(predict.mse())

11.365736624587639

In [18]:
print('R^2 train: %.3f, test: %.3f' % (
        glm_model.r2(valid=True),
        predict.r2()))

R^2 train: 0.575, test: 0.527


In [14]:
h2o.shutdown()#Cerramos la sesion del H2O

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  """Entry point for launching an IPython kernel.


H2O session _sid_905d closed.


## CASO PRACTICO: aprobación presidencial

In [19]:
import pandas as pd
df = pd.read_csv('D:/Python/2. Nivel II/5/datos/Aprobacion.csv', encoding="latin_1", sep=";")
df.head()

,Año,Mes,IPC,Inflación,SalariosReales,CrisisDeGabinete,LunaDeMiel1,LunaDeMiel2,AñoElectoral,Aprobaciónn,Desaprobación,No precisa,T-1
0,2006,Agosto,90.261885,0.139281,247.00,0,1,0,0,63,16,21,63
1,2006,Septiembre,90.286647,0.027433,246.93,0,1,0,0,58,21,21,63
2,2006,Octubre,90.326182,0.043789,246.82,0,1,0,0,57,24,19,58
3,2006,Noviembre,90.071323,-0.282155,247.52,0,1,0,0,55,22,23,57
4,2006,Diciembre,90.094572,0.025811,247.45,0,1,0,0,52,30,18,55


###### Descripción de variables:

- TARGET: Aprobacion
- X1: Salarios Reales
- X2: Inflación
- X3: IPC (índice de precio al consumidor)

Ejercicio:

- 1. realizar un análisis exploratorio de las variables predictoras con respecto al target (matriz de dispersión y mapa de calor de las correlaciones).
- 2. Construir los datos de entrenamiento y de prueba, teniendo en cuenta que 70% de los datos es para entrenamoento y 30% de prueba.
- 3. Realizar un modelo de regresión lineal simple: TARGET ~ X1
- 4. Realizar un modelo de regresión lineal múltiple: TARGET ~ X1 +X2 +X3
- Evaluar el modelo: Raíz cuadra del error cuadrático medio y $R^2$ tanto para el modelo lineal simple como para el múltiple.

- Enviar: python@socialdata-peru.con
- Asunto: Laboratorio regresión lineal